## 加上羊群指標

In [1]:
import finlab

In [2]:
finlab.login(api_token='ID0hv1JVLP3bs86Qg/xslT1fwKjN0bclQVRCH7eQiAYh3e5/XNh77u3zaPDOOGJQ#vip_m')

輸入成功!


In [3]:
from finlab import data
from finlab.backtest import sim
import pandas as pd
import numpy as np
from finlab import data
from finlab.backtest import sim
import pandas as pd
import numpy as np
from finlab import backtest

In [4]:
from finlab import data
# with data.universe(market='TSE_OTC'): 
with data.universe(market ='STOCK_FUTURE'):

    close = data.get('price:收盤價')

    # treemap data

    # marketcap - size
    basic_info = data.get('company_basic_info')

    basic_info.index = basic_info.stock_id
    marketcap = (basic_info['實收資本額(元)'] / 10) * close.iloc[-1]

    # stock category
    category = basic_info['產業類別']

    # stock id and name
    name = basic_info['stock_id'] + basic_info['公司簡稱']
    
    bt = data.get('broker_transactions')
    bt.head()

 Use "pip install finlab==0.4.4" to update the latest version.


In [11]:
broker_volume = (bt
    .reset_index() # reset index
    .groupby(['date', 'stock_id']).sum() # group stock and date
    .reset_index()
    .pivot(columns='stock_id', index='date')
)

broker_volume

broker                                                        \
stock_id       0050     0056     0061   006205   006206   006207    00636   
date                                                                        
2018-01-02  16203.0  12302.0  13954.0  13893.0  13388.0  14164.0  12885.0   
2018-01-03  12768.0  12482.0  12746.0  14328.0  11385.0  12139.0  15079.0   
2018-01-04  13562.0  11337.0  11696.0  13938.0  12228.0  12257.0  13594.0   
2018-01-05  15490.0  10887.0  13855.0  14632.0  12485.0  13637.0  14570.0   
2018-01-08  14764.0  15806.0  10969.0  12941.0   9870.0  14234.0  12968.0   
...             ...      ...      ...      ...      ...      ...      ...   
2023-06-12  13653.0  15058.0  12547.0  15643.0   9310.0   7625.0  12793.0   
2023-06-13  13820.0  15138.0  13778.0  13600.0  10615.0   8417.0  12248.0   
2023-06-14  13451.0  13705.0  14914.0  14152.0  11962.0   6121.0  14826.0   
2023-06-15  13313.0  12956.0  13044.0  12502.0  14181.0  12905.0  13882.0   
2023-06-16  12981.0  13193.0  11719.0  12523.0  11748.0  11472.0  12660.0   

                                       ...    sell                        \
stock_id      00639    00643    00878  ...    8299    8358   8436   8454   
date                                   ...                                 
2018-01-02  15166.0  14784.0      NaN  ...   359.0  1086.0  115.0  103.0   
2018-01-03  12720.0  13334.0      NaN  ...   412.0  1237.0  284.0   99.0   
2018-01-04  13293.0  12205.0      NaN  ...   458.0  1212.0  434.0   93.0   
2018-01-05  13517.0  15699.0      NaN  ...   424.0  1967.0  200.0  124.0   
2018-01-08  12913.0  12213.0      NaN  ...  1301.0  4940.0  135.0  117.0   
...             ...      ...      ...  ...     ...     ...    ...    ...   
2023-06-12  12854.0  12980.0  16515.0  ...   979.0  4695.0  332.0  239.0   
2023-06-13  11930.0  13221.0  14507.0  ...  3137.0  3379.0  657.0  273.0   
2023-06-14  10473.0  12135.0  13684.0  ...  1257.0  1411.0  790.0  611.0   
2023-06-15  12025.0  13404.0  13421.0  ...  1030.0  1534.0  328.0  202.0   
2023-06-16  13254.0  10541.0  12821.0  ...   674.0   965.0  312.0  312.0   

                                                          
stock_id     8942    9904   9914    9938    9939    9945  
date                                                      
2018-01-02   96.0  2542.0  821.0   446.0   844.0  1085.0  
2018-01-03  132.0  2641.0  306.0   574.0   810.0  1566.0  
2018-01-04  232.0  2967.0  474.0   674.0   614.0  1654.0  
2018-01-05  129.0  3104.0  296.0   692.0   566.0  2158.0  
2018-01-08   59.0  3931.0  384.0   288.0   631.0  1673.0  
...           ...     ...    ...     ...     ...     ...  
2023-06-12  356.0  2005.0  547.0   198.0  1487.0  2267.0  
2023-06-13  190.0  3090.0  685.0   372.0  2231.0  2972.0  
2023-06-14  205.0  3828.0  707.0   521.0  1610.0  3083.0  
2023-06-15  188.0  4006.0  422.0  1999.0  1742.0  1320.0  
2023-06-16  452.0  7645.0  559.0  1355.0  1682.0  6541.0  

[1329 rows x 699 columns]

In [ ]:
broker_force = (broker_volume.buy - broker_volume.sell) / (broker_volume.buy + broker_volume.sell)
broker_force

stock_id,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,0.035112,-0.104948,0.101500,-0.105175,0.005587,0.026798,-0.050239,-0.030033,-0.028571,-0.186386,...,0.000000,0.033408,0.134454,0.000000,NaN,-0.151786,-0.002283,-0.064758,0.000000,0.000000
2018-01-03,0.103017,0.011817,0.048101,-0.152900,-0.104839,0.043601,0.000000,0.014224,-0.003922,-0.346559,...,-0.002755,0.190907,0.051617,0.000000,NaN,-0.084399,0.019368,-0.160305,0.121212,0.000000
2018-01-04,0.038813,0.060391,-0.026265,-0.043880,-0.029240,-0.017182,0.000000,0.092251,-0.005525,-0.009097,...,0.000000,0.153748,-0.100985,0.020408,NaN,0.126437,0.001745,-0.252425,0.000000,0.000000
2018-01-05,-0.046917,0.167435,0.104506,-0.096436,-0.094225,-0.037500,0.000000,0.075521,-0.007634,-0.109649,...,0.000000,0.210247,-0.065723,0.000000,NaN,-0.116564,0.038251,0.164018,0.049383,0.000000
2018-01-08,0.113996,0.150341,-0.221683,-0.145414,-0.186944,0.031847,-0.010753,0.142982,-0.026087,0.100629,...,-0.045977,0.250280,0.004843,0.000000,NaN,0.075862,0.075159,-0.013662,-0.065327,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-23,-0.160566,0.091126,0.062374,0.000000,0.006319,0.017751,0.006608,0.103797,0.000000,0.185765,...,-0.068182,0.168764,-0.070140,-0.047619,0.000000,0.081633,0.111421,-0.317397,0.000000,0.030303
2023-05-24,-0.176969,-0.033753,0.117318,0.109728,0.012128,0.087629,0.007092,0.037918,0.000000,0.059948,...,0.082452,0.071124,-0.119266,-0.041237,0.064935,0.085714,0.028148,-0.373635,0.000000,-0.083585
2023-05-25,-0.099260,-0.049039,-0.067751,-0.282671,-0.276495,-0.029586,0.042296,-0.097345,0.000000,-0.141093,...,-0.184116,-0.300061,0.057143,-0.058824,0.062992,0.031250,-0.262765,-0.148361,0.000000,-0.071756


In [ ]:
# broker_force - color
broker_force_col = broker_force.iloc[-1]

In [ ]:
import pandas as pd
treemap_data = (
    pd.concat([name, category, marketcap, broker_force_col], axis=1)
    .set_axis(['name', 'category', 'marketcap', 'broker_force'], axis=1)
    .dropna()
)

treemap_data

,name,category,marketcap,broker_force
1101,1101台泥,水泥工業,2.806383e+11,-0.056700
1102,1102亞泥,水泥工業,1.554889e+11,-0.018521
1103,1103嘉泥,水泥工業,1.545687e+10,0.042735
1104,1104環泥,水泥工業,1.820302e+10,-0.145444
1108,1108幸福,水泥工業,6.152018e+09,0.010459
...,...,...,...,...
9951,9951皇田,電機機械,5.475190e+09,0.014184
9955,9955佳龍,其他,2.595687e+09,-0.153989
9958,9958世紀鋼,鋼鐵工業,2.544741e+10,0.244607
9960,9960邁達康,貿易百貨,1.041368e+09,0.000000


In [ ]:
import plotly.express as px
import numpy as np


fig = px.treemap(treemap_data, path=['category', 'name'], values='marketcap',
                  color='broker_force', hover_data=['broker_force'],
                  color_continuous_scale='Temps',
                  color_continuous_midpoint=0, height=800, title='券商分點籌碼流動')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [12]:
with data.universe(market='TSE_OTC'): 
    broker = data.get('broker_transactions')

    buy_sell = broker.groupby(['stock_id', 'date']).sum().reset_index().pivot('date', 'stock_id')
    buy_vol = buy_sell['buy'].fillna(0)
    sell_vol = buy_sell['sell'].fillna(0)

    force = (buy_vol - sell_vol) / (buy_vol + sell_vol)
    force = force[force.columns[~force.columns.str[-1].isin(['L', 'B'])]]

TypeError: DataFrame.pivot() takes 1 positional argument but 3 were given

In [ ]:
force

stock_id,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,0.035112,-0.104948,0.101500,-0.105175,0.005587,0.026798,-0.050239,-0.030033,-0.028571,-0.186386,...,0.000000,0.033408,0.134454,0.000000,NaN,-0.151786,-0.002283,-0.064758,0.000000,0.000000
2018-01-03,0.103017,0.011817,0.048101,-0.152900,-0.104839,0.043601,0.000000,0.014224,-0.003922,-0.346559,...,-0.002755,0.190907,0.051617,0.000000,NaN,-0.084399,0.019368,-0.160305,0.121212,0.000000
2018-01-04,0.038813,0.060391,-0.026265,-0.043880,-0.029240,-0.017182,0.000000,0.092251,-0.005525,-0.009097,...,0.000000,0.153748,-0.100985,0.020408,NaN,0.126437,0.001745,-0.252425,0.000000,0.000000
2018-01-05,-0.046917,0.167435,0.104506,-0.096436,-0.094225,-0.037500,0.000000,0.075521,-0.007634,-0.109649,...,0.000000,0.210247,-0.065723,0.000000,NaN,-0.116564,0.038251,0.164018,0.049383,0.000000
2018-01-08,0.113996,0.150341,-0.221683,-0.145414,-0.186944,0.031847,-0.010753,0.142982,-0.026087,0.100629,...,-0.045977,0.250280,0.004843,0.000000,NaN,0.075862,0.075159,-0.013662,-0.065327,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-23,-0.160566,0.091126,0.062374,0.000000,0.006319,0.017751,0.006608,0.103797,0.000000,0.185765,...,-0.068182,0.168764,-0.070140,-0.047619,0.000000,0.081633,0.111421,-0.317397,0.000000,0.030303
2023-05-24,-0.176969,-0.033753,0.117318,0.109728,0.012128,0.087629,0.007092,0.037918,0.000000,0.059948,...,0.082452,0.071124,-0.119266,-0.041237,0.064935,0.085714,0.028148,-0.373635,0.000000,-0.083585
2023-05-25,-0.099260,-0.049039,-0.067751,-0.282671,-0.276495,-0.029586,0.042296,-0.097345,0.000000,-0.141093,...,-0.184116,-0.300061,0.057143,-0.058824,0.062992,0.031250,-0.262765,-0.148361,0.000000,-0.071756


In [ ]:
pos = (force.is_largest(1)).hold_until((force.rolling(3).sum() < 0), rank=force, nstocks_limit=10)

report = backtest.sim(pos,  resample='W', fee_ratio=1.425/1000, tax_ratio=3/1000*0.3, name="當天分點買超最大", upload=True)
print(report.get_stats(resample='1d', riskfree_rate=0.02)['daily_sharpe'])
print(report.get_stats(resample='1d', riskfree_rate=0.02)['max_drawdown'])
report.display()
report.display_mae_mfe_analysis()

1.3330001870605181
-0.28161655417151565


--Stats--
annualized_rate_of_return     31.91%
sharpe                          1.33
max_drawdown                 -28.16%
win_ratio                     48.61%

--Latest Position--


stock_id
2330    0.25
2344    0.25
3189    0.25
6191    0.25
Name: 2023-06-04 00:00:00, dtype: float64

In [ ]:
report.trades['return'].max()

0.8151260504201681